In [1]:
import pypostmaster
import getpass
from queryrunner_client import *
from queryrunner_client import QueryRunnerException
import numpy as np
import pandas as pd
from pandas.io import gbq
import pytz
from datetime import datetime, timedelta, tzinfo
import time
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.relativedelta import relativedelta
import docx
from docx import Document
from docxtpl import DocxTemplate, InlineImage
from docx.shared import Inches
import jinja2
import math
import yaml
import os
import signal

import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import uuid
import json
from google.cloud import bigquery
import base64
import nbformat
from IPython import get_ipython
import shutil
import subprocess
import random
from croniter import croniter


ipython = get_ipython()

In [2]:
from LP_helper import ymlParser 
from LP_helper import validateDriveAccess
from LP_helper import runBQuery
from LP_helper import getNextID
from LP_helper import getDateFromShortName
from LP_helper import getDqrRequest
from LP_helper import isDqrRequestAvailable
from LP_helper import executeNotebookCell
from LP_helper import postDqrCompleted
from LP_helper import sendDqrMail
from LP_helper import postDqrScriptFail
from LP_helper import CreateDqrLogger
from LP_helper import resetDqrLongRunningTask
from LP_helper import maxDqrTaskChecker
from LP_helper import resetDqrForTesting
from LP_helper import postDqrAutoLog
from LP_helper import Fail_mail


from LP_helper import uploadFileToDrive
from LP_helper import getQueryUUID
from LP_helper import getEncodedImg

from LP_helper import getDpvQueryReportID
from LP_helper import getDataPushRequest
from LP_helper import triggerDataPush
from LP_helper import getDataPushReportRequest
from LP_helper import postDpvCompleted
from LP_helper import sendDpvMail

from LP_helper import getHistoricalScheduler
from LP_helper import isScheduled
from LP_helper import nextRunin
from LP_helper import SchedulesToPrecheck
from LP_helper import getThisWeekStartEnd
from LP_helper import getSchedulerForPreCheck
from LP_helper import postReportRequest
from LP_helper import postSchedulerToDqrRequest

from LP_helper import getData

# Schedule to Pre-Check

In [3]:
historicalScheduler = getHistoricalScheduler()
df = pd.DataFrame(historicalScheduler)
df

,scheduler_id,created_at,updated_at,last_executed_at,is_active,dqr_name,run_on,run_condition,run_status,email_address,query_run_condition,getRunId_querybuilder,cron_minute,cron_hour,cron_day,cron_month,cron_weekday,dashboard,parameters,cronEcpresssion
0,3,2024-05-14 14:36:45,2024-12-12 08:02:36,2024-12-12 13:32:35,1,France Rides Old,Any,Recurring; Until Complete,None,mnizam1@ext.uber.com,Fresh Run all,None,0,12,*,*,4,gi_gss,None,0 12 * * 4
1,84,2024-11-21 08:47:34,2024-12-12 08:02:37,2024-12-12 13:32:36,1,US Eats Ensemble,mnizam1,Recurring;Until Complete,None,mnizam1@ext.uber.com,Run by UUID if a fresh run is available within...,uCidl3haP,0,13,*,*,4,gi_gss,None,0 13 * * 4
2,86,2024-11-21 13:30:31,2024-12-12 08:05:44,2024-12-12 13:35:43,1,US Eats Ensemble,vjayap,Recurring;Until Complete,None,vjayap@ext.uber.com,Fresh Run all,uCidl3haP,35,13,*,*,4,gi_gss,None,35 13 * * 4
3,100,2024-12-10 10:33:28,2024-12-10 11:35:16,2024-12-10 17:05:15,1,Australia Eats,kshiva19,Once,None,kshiva19@ext.uber.com,Run by UUID if a fresh run is available within...,,3,17,*,*,2,gi_gss,"{""query_id"": """"}",3 17 * * 2
4,101,2024-12-10 17:12:26,2024-12-11 11:20:02,2024-12-11 16:50:01,1,US Eats Ensemble,kshiva19,Once,None,kshiva19@ext.uber.com,Fresh Run all,uCidl3haP,50,16,*,*,3,gi_gss,"{""query_id"": ""uCidl3haP""}",50 16 * * 3
5,106,2024-12-16 10:25:58,2024-12-16 04:57:04,2024-12-16 10:27:02,1,Testing,kshiva19,Once,None,kshiva19@ext.uber.com,Run by UUID if a fresh run is available within...,None,27,10,*,*,1,test,"{""text1"": ""launchpad"", ""text2"": ""launchpad par...",27 10 * * 1


In [6]:
for schedule in historicalScheduler:
    status = isScheduled(schedule['last_executed_at'], schedule['cronEcpresssion'])
    
    if status: 
        print(schedule)
        SchedulesToPrecheck(schedule)
    print(status, schedule['dqr_name'],schedule['dashboard'])
    dashboard = schedule['dashboard']
    nextRunin(schedule['cronEcpresssion'])
    

False France Rides Old gi_gss
3 days, 1:31:37.861584
False US Eats Ensemble gi_gss
3 days, 2:31:37.861078
False US Eats Ensemble gi_gss
3 days, 3:06:37.860563
False Australia Eats gi_gss
1 day, 6:34:37.860066
False US Eats Ensemble gi_gss
2 days, 6:21:37.859554
False Testing test
6 days, 23:58:37.859046


# Pre-Check to DQR request

In [24]:
onPreCheck = getSchedulerForPreCheck()
onPreCheck



            SELECT * FROM launchpad_schedular_on_pre_check_dev
            WHERE run_status != 'Completed'
            AND (
                past_checked_at IS NULL OR 
                past_checked_at < '2024-12-16 10:07:06'
            )
        
   pre_check_id  scheduler_id          created_at          updated_at  \
0           184           106 2024-12-16 04:57:03 2024-12-16 04:57:03   

  getRunId_querybuilder run_status runIds past_checked_at  \
0                  None                              None   

                                          parameters  
0  {"text1": "launchpad", "text2": "launchpad par...  


[{'pre_check_id': 184,
  'scheduler_id': 106,
  'created_at': Timestamp('2024-12-16 04:57:03'),
  'updated_at': Timestamp('2024-12-16 04:57:03'),
  'getRunId_querybuilder': None,
  'run_status': '',
  'runIds': '',
  'past_checked_at': None,
  'parameters': '{"text1": "launchpad", "text2": "launchpad parameter", "c_code": "Hh9w3g6hR", "c_name": "onEk0k6nx"}'}]

In [7]:
# from pandas import Timestamp

# onPreCheck = [{'pre_check_id': 184,
#   'scheduler_id': 106,
#   'created_at': Timestamp('2024-12-16 04:57:03'),
#   'updated_at': Timestamp('2024-12-16 04:57:03'),
#   'getRunId_querybuilder': None,
#   'run_status': '',
#   'runIds': '',
#   'past_checked_at': None,
#   'parameters': '{"text1": "launchpad", "text2": "launchpad parameter", "c_code": "Hh9w3g6hR", "c_name": "onEk0k6nx"}'}]

In [8]:
list(map(lambda x: postSchedulerToDqrRequest(x, dashboard), onPreCheck))


param keys: ['text1', 'text2', 'c_code', 'c_name']
  variable_name  exec_type      column_name
0        c_name  qb_report  competitor_name
1        c_code  qb_report     country_code
2         text1       text             None
3         text2       text             None
text_params = {'text1': 'launchpad', 'text2': 'launchpad parameter'}
qb_report_params = {'c_code': 'Hh9w3g6hR', 'c_name': 'onEk0k6nx'}
Decoded Text Parameters: {'text1': 'launchpad', 'text2': 'launchpad parameter'}


12/16/2024 04:58:30 AM  Obtained utoken email from file 
12/16/2024 04:58:30 AM  Obtained utoken from utoken cli 
12/16/2024 04:58:30 AM  Could not obtain utoken from cli: b'/bin/sh: 1: usso: not found\n'
Trying to obtain from file 
12/16/2024 04:58:30 AM  Obtained utoken for user 
12/16/2024 04:58:30 AM  Fetching metadata for Report Hh9w3g6hR 
12/16/2024 04:58:30 AM  Loaded object metadata. 
12/16/2024 04:58:30 AM Send tier_metadata {'report_id': 'Hh9w3g6hR'} to Queryrunner V2.


Query executed successfully.


12/16/2024 04:58:30 AM  [Polling] 5ce68005-a8ee-45e0-92c4-bcf44fb3a969 
12/16/2024 04:58:30 AM  [Status] created 
12/16/2024 04:58:31 AM  [Status] started auth check 
12/16/2024 04:58:32 AM  [Status] started execution 
12/16/2024 04:58:36 AM  [Status] completed success 
12/16/2024 04:58:36 AM  [Query Success] completed success 
12/16/2024 04:58:37 AM  Obtained utoken email from file 
12/16/2024 04:58:37 AM  Obtained utoken from utoken cli 
12/16/2024 04:58:37 AM  Could not obtain utoken from cli: b'/bin/sh: 1: usso: not found\n'
Trying to obtain from file 
12/16/2024 04:58:37 AM  Obtained utoken for user 
12/16/2024 04:58:37 AM  Fetching metadata for Report onEk0k6nx 
12/16/2024 04:58:37 AM  Loaded object metadata. 
12/16/2024 04:58:37 AM Send tier_metadata {'report_id': 'onEk0k6nx'} to Queryrunner V2.


Query executed successfully.


12/16/2024 04:58:37 AM  [Polling] 705d777a-222f-42b7-a1e2-258f8a66844f 
12/16/2024 04:58:37 AM  [Status] created 
12/16/2024 04:58:38 AM  [Status] finished auth check 
12/16/2024 04:58:39 AM  [Status] started execution 
12/16/2024 04:58:47 AM  [Status] completed success 
12/16/2024 04:58:47 AM  [Query Success] completed success 


status: Completed


json param:{&quot;text1&quot;: &quot;launchpad&quot;, &quot;text2&quot;: &quot;launchpad parameter&quot;, &quot;c_code&quot;: &quot;\\N, NOR, SVK, VEN, SGP, BRB, GHA, USA, EST, FRA, ECU, KOR, AUS, JOR, PRT, JPN, NIC, CHE, SAU, DNK, ESP, NGA, ARE, HRV, AUT, SLV, SVN, FIN, CIV, ARG, UGA, LBN, DOM, URY, PRY, GTM, ZAF, SWE, MEX, REU, PAK, MAC, GBR, GRC, US, UKR, HUN, PAN, MAR, LTU, ALA, ITA, GB, JAM, NPL, TTO, TUR, CAN, POL, TZA, BOL, SRB, BRA, MUS, NLD, BGD, LKA, CHL, BEL, IRL, HKG, QAT, NZL, PER, ROU, IND, COL, HND, TWN, CZE, DEU, BHR, KEN, EGY, GLP, CRI, KWT&quot;, &quot;c_name&quot;: &quot;Bite Squad, KFC, Mr. Delivery, 99, HelloFresh, Pizza Hut, OrderUp, Costco, Deliveroo, Waymo, Bolt, My Taxi, Go-Jek, inDrive, Domino&#x27;s Pizza, Free Now, Yelp, Yandex, Walmart, Ride Austin, Subway, Dunkin&#x27; Donuts, Uber Pass, Starbucks, Fresh Direct, Yango, Drizly, DemaeCan, DoorDash, Good Eggs, Order Up, Rite Aid, Bird, Wolt, Ritual, 5cd16a998fa0e3b107bf919b453e4d8e7c2a8865

[True]

In [8]:
# list(map(postSchedulerToDqrRequest, onPreCheck,dashboard))

In [ ]:
parameters = {'query_id': 'uCidl3haP'}
param_type = {'query_id': 'qb_report'}

In [ ]:
param_type = {'query_id': ['qb_report','run_id']}